# Dataset Download

In [ ]:
!gdown 1FnF607sBYWaaLd7YyRDKLcBM2OQghDCp

Downloading...
From: https://drive.google.com/uc?id=1FnF607sBYWaaLd7YyRDKLcBM2OQghDCp
To: /content/archive.zip
100% 346M/346M [00:01<00:00, 179MB/s]


In [ ]:
from zipfile import ZipFile

In [ ]:
with ZipFile('/content/archive.zip', 'r') as zipObj:
	zipObj.extractall('/content/')

In [11]:
import os
import cv2
import numpy as np

Data = "/content/data/train/"
Classes = ["close eyes", "open eyes"]

X = []
y = []
data_limit = 24000
img_size = 64

def create_training_data():
    count = 0
    category_count = [0, 0]
    for category in Classes:
        path = os.path.join(Data, category)
        class_num = Classes.index(category)
        for img in os.listdir(path):
            try:
                if count >= data_limit:
                    return

                if category_count[class_num] >= data_limit // 2:
                    continue

                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (img_size, img_size))
                new_array = np.expand_dims(new_array, axis=-1)

                X.append(new_array)
                y.append(class_num)

                count += 1
                category_count[class_num] += 1
            except Exception as e:
                print(e)

create_training_data()

X = np.array(X)
X = X / 255.0
X = (np.array(X) - np.min(X)) / (np.max(X) - np.min(X))
Y = np.array(y)

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Conv2D, BatchNormalization, MaxPooling2D,Dropout, Flatten

model = tf.keras.models.Sequential([
      Input(shape=(64, 64, 1)),

      Conv2D(filters = 32, kernel_size = 5, strides = 1, activation = 'relu'),
      Conv2D(filters = 32, kernel_size = 5, strides = 1, activation = 'relu', use_bias=False),
      BatchNormalization(),
      MaxPooling2D(strides = 2),
      Dropout(0.3),

      Conv2D(filters = 64, kernel_size = 3, strides = 1, activation = 'relu'),
      Conv2D(filters = 64, kernel_size = 3, strides = 1, activation = 'relu', use_bias=False),
      BatchNormalization(),
      MaxPooling2D(strides = 2),
      Dropout(0.3),

      Flatten(),
      Dense(units  = 256, activation = 'relu', use_bias=False),
      BatchNormalization(),

      Dense(units = 128, use_bias=False, activation = 'relu'),

      Dense(units = 84, use_bias=False, activation = 'relu'),
      BatchNormalization(),
      Dropout(0.3),

      Dense(units = 1, activation = 'sigmoid')
  ])

In [13]:
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [15]:
callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='model.h5',
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose =1)
model.fit(x_train, y_train, validation_split=0.2, epochs=30, batch_size=32, callbacks=callback)

Epoch 1/30
398/402 [============================>.] - ETA: 0s - loss: 0.2399 - accuracy: 0.9066
Epoch 1: val_loss improved from inf to 0.46950, saving model to model.h5
402/402 [==============================] - 10s 14ms/step - loss: 0.2403 - accuracy: 0.9063 - val_loss: 0.4695 - val_accuracy: 0.7764
Epoch 2/30
401/402 [============================>.] - ETA: 0s - loss: 0.1281 - accuracy: 0.9523
Epoch 2: val_loss improved from 0.46950 to 0.37449, saving model to model.h5
402/402 [==============================] - 6s 16ms/step - loss: 0.1279 - accuracy: 0.9524 - val_loss: 0.3745 - val_accuracy: 0.8212
Epoch 3/30
400/402 [============================>.] - ETA: 0s - loss: 0.1006 - accuracy: 0.9653
Epoch 3: val_loss did not improve from 0.37449
402/402 [==============================] - 6s 14ms/step - loss: 0.1006 - accuracy: 0.9653 - val_loss: 0.9271 - val_accuracy: 0.7425
Epoch 4/30
402/402 [==============================] - ETA: 0s - loss: 0.0894 - accuracy: 0.9691
Epoch 4: val_loss did 

In [10]:
model.evaluate(x_test, y_test)

83/83 [==============================] - 0s 5ms/step - loss: 0.1504 - accuracy: 0.9436


[0.15037082135677338, 0.9435606002807617]